# User Categories

Categoriasation of users according to the categories of the points of interest they visit.

In [1]:
import pandas as pd
import datetime

### Data

In [2]:
walkins = pd.read_csv('Aueb Dmst Assignment Data\walkins.csv', dtype = str)
users = pd.read_csv('Aueb Dmst Assignment Data\\users.csv', dtype = str)
pois = pd.read_csv('Aueb Dmst Assignment Data\pois.csv', dtype = str)
categories = pd.read_csv('Aueb Dmst Assignment Data\categories.csv', dtype = str)
points = pd.read_csv('Aueb Dmst Assignment Data\\users.csv', dtype = str)

In [3]:
categories.rename(columns = {'id': 'category_id', 'title': 'category'}, inplace = True)
pois.rename(columns = {'id': 'poi_id'}, inplace = True)
pois_categories = pd.merge(pois[['poi_id', 'category_id', 'title']],
                           categories,
                           how = 'left',
                           on = 'category_id')

### User Categorisation

In [4]:
visits = pd.merge(walkins[['created', 'poi_id', 'user_id']],
                  pois_categories[['poi_id', 'category_id', 'category']],
                  how = 'left',
                  on = 'poi_id')
visits = visits[['created', 'user_id', 'category']]

In [5]:
users = visits.groupby(by=['user_id', 'category']).count().reset_index()
users.rename(columns = {'created': 'visits'}, inplace = True)

#### Woman

Users belong to this category if they fulfill at least one of the following:
* Visits in Beauty Stores > 2
* Visits in Women's Stores > 2

In [6]:
users['Woman'] = ((users['category'] == 'Women\'s Store') & (users['visits'] > 2)) | ((users['category'] == 'Beauty Store') & (users['visits'] > 2))


#### Parent

Users belong to this category if they fulfill at least one of the following:
* Visits in Playgrounds > 1
* Visits in Kids Stores > 2

In [7]:
users['Parent'] = ((users['category'] == 'Playground') & (users['visits'] > 3)) | ((users['category'] == 'Kids Store') & (users['visits'] > 2))

#### Pet Owner

Users belong to this category if they fulfill at least one of the following:
* Visits in Pet Stores > 2

In [8]:
users['Pet Owner'] = ((users['category'] == 'Pet Store') & (users['visits'] > 2))

#### Gamer

Users belong to this category if they fulfill at least one of the following:
* Visits in Gaming Cafes > 1

In [9]:
users['Gamer'] = ((users['category'] == 'Gaming Cafe') & (users['visits'] > 1))

#### Student

Users belong to this category if they fulfill at least one of the following:
* Visits in University > 1

In [10]:
users['Student'] = ((users['category'] == 'University') & (users['visits'] > 1))

#### Athletic

Users belong to this category if they fulfill at least one of the following:
* Visits in Gyms > 3

In [11]:
users['Athletic'] = ((users['category'] == 'Gym') & (users['visits'] > 3))

#### Foodie

Categoies of Points of Interest that fall into this category: Souvlaki, Taverna, Steakhouse, Italian, Restaurant, Ethnic, Burgers, Sandwich Place

Users belong to this category if they fulfill at least one of the following:
* Variety of categories visited > 3

In [12]:
users_foodie = users[(users['category'] == 'Souvlaki') |
                     (users['category'] == 'Taverna') |
                     (users['category'] == 'Steakhouse') |
                     (users['category'] == 'Italian') |
                     (users['category'] == 'Restaurant') |
                     (users['category'] == 'Ethnic') |
                     (users['category'] == 'Burgers') |
                     (users['category'] == 'Sandwich Place')]
users_foodie = users_foodie.groupby(by=['user_id']).count().reset_index()

In [13]:
users_foodie['variety'] = users_foodie['category'] > 3

In [14]:
users = pd.merge(users,
                 users_foodie[['user_id', 'variety']],
                 on = 'user_id',
                 how = 'left')

In [15]:
users['Foodie'] = False
users['Foodie'] = (users['variety'] == True)

#### Shopper

* Categoies of Points of Interest that fall into this category: Women's Store, Mobile Phone Shop, Shoe Store, Clothing Store, Beauty Store, Game Store, Electronics Store, Kids Store, Shopping Mall, Home Store

Users belong to this category if they fulfill at least one of the following:
* Variety of categories visited > 4

In [16]:
users_shopper = users[(users['category'] == ' Women\'s Store') |
                      (users['category'] == 'Mobile Phone Shop') |
                      (users['category'] == 'Shoe Store') |
                      (users['category'] == 'Clothing Store') |
                      (users['category'] == 'Beauty Store') |
                      (users['category'] == 'Game Store') |
                      (users['category'] == 'Electronics Store') | 
                      (users['category'] == 'Kids Store') |
                      (users['category'] == 'Shopping Mall') |
                      (users['category'] == 'Home Store')]
users_shopper = users_shopper.groupby(by=['user_id']).count().reset_index()

In [17]:
users_shopper['variety_2'] = users_shopper['category'] > 4

In [18]:
users = pd.merge(users,
                 users_shopper[['user_id', 'variety_2']],
                 on = 'user_id',
                 how = 'left')

In [19]:
users['Shopper'] = False
users['Shopper'] = (users['variety_2'] == True)

#### Final DataFrame

In [20]:
no_category = users[(users['Woman'] != True) & (users['Parent'] != True) & (users['Pet Owner'] != True) &
                    (users['Gamer'] != True) & (users['Student'] != True) & (users['Athletic'] != True) & 
                    (users['Foodie'] != True) & (users['Shopper'] != True)]
users = users[(users['Woman'] == True) | (users['Parent'] == True) | (users['Pet Owner'] == True) |
              (users['Gamer'] == True) | (users['Student'] == True) | (users['Athletic'] == True) | 
              (users['Foodie'] == True) | (users['Shopper'] == True)]
user_categories = users[['user_id', 'Woman', 'Parent', 'Pet Owner', 'Gamer', 'Student', 'Athletic', 'Foodie', 'Shopper']]
user_categories.drop_duplicates(inplace = True)
user_categories.replace({False: 0, True: 1}, inplace=True)
user_categories = user_categories.groupby(by=['user_id']).sum()

<ipython-input-20-e54053979ae9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_categories.drop_duplicates(inplace = True)
C:\Python\lib\site-packages\pandas\core\frame.py:4521: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


We only want values 0 and 1, where 0 is false and 1 is true.

In [21]:
user_categories.Foodie.unique()

array([0, 4, 3, 5, 6, 1], dtype=int64)

In [22]:
user_categories.Shopper.unique()

array([2, 4, 6, 3, 5, 0, 1], dtype=int64)

In [23]:
user_categories['Foodie'] = user_categories['Foodie'].replace([4, 3, 5, 6], 1)
user_categories['Shopper'] = user_categories['Shopper'].replace([2, 4, 6, 3, 5], 1)

In [24]:
user_categories.head(10)

,Woman,Parent,Pet Owner,Gamer,Student,Athletic,Foodie,Shopper
user_id,,,,,,,,
10,0,0,0,0,1,0,0,1
100,1,0,0,0,1,1,0,1
101,1,0,0,0,1,1,0,1
103,1,1,1,0,1,1,0,1
104,1,0,0,0,1,0,0,1
105,1,0,1,0,1,0,0,1
106,1,0,1,0,1,0,1,1
107,1,0,1,0,1,1,0,1
108,1,0,0,0,1,1,0,1


### Amount of users in each User Category

In [25]:
user_categories.sum().reset_index()

,index,0
0,Woman,707
1,Parent,46
2,Pet Owner,360
3,Gamer,27
4,Student,693
5,Athletic,346
6,Foodie,41
7,Shopper,689


### Users that do not belong in any of the User Categories

In [26]:
no_category.head(5)

,user_id,category,visits,Woman,Parent,Pet Owner,Gamer,Student,Athletic,variety,Foodie,variety_2,Shopper
46,10037,Supermarket,1,False,False,False,False,False,False,NaN,False,NaN,False
47,10079,Home Store,1,False,False,False,False,False,False,NaN,False,False,False
75,10163,Cafe,4,False,False,False,False,False,False,NaN,False,NaN,False
76,10163,Supermarket,7,False,False,False,False,False,False,NaN,False,NaN,False
77,10185,Supermarket,1,False,False,False,False,False,False,NaN,False,NaN,False


In [27]:
no_category.groupby(by = 'category').count().sort_values(by = 'user_id', ascending = False).head(10)

,user_id,visits,Woman,Parent,Pet Owner,Gamer,Student,Athletic,variety,Foodie,variety_2,Shopper
category,,,,,,,,,,,,
Supermarket,171,171,171,171,171,171,171,171,43,171,68,171
Park,130,130,130,130,130,130,130,130,23,130,49,130
Square,115,115,115,115,115,115,115,115,35,115,66,115
Metro Station,112,112,112,112,112,112,112,112,34,112,64,112
Cultural Center,108,108,108,108,108,108,108,108,22,108,43,108
Beach,103,103,103,103,103,103,103,103,19,103,39,103
Marina,87,87,87,87,87,87,87,87,12,87,24,87
Cafe,71,71,71,71,71,71,71,71,38,71,51,71
Landmark,63,63,63,63,63,63,63,63,4,63,23,63


In [28]:
no_category.groupby(by = 'user_id').count().sort_values(by = 'category', ascending = False).value_counts('category')

category
1     108
7      46
2      34
4      25
5      18
6      15
3      12
10      9
13      9
8       5
9       5
22      5
15      5
18      4
17      2
21      2
12      2
14      2
11      2
16      1
20      1
23      1
dtype: int64

**We reach the conclusion that there are no common visits by these users and that is the reason we cannot include them in any of the categories or create any new ones.**

### Overlapping categories

Checking the number of users of each User Category that, at the same time, belong to other User Categories.

#### Woman

In [29]:
user_categories[user_categories['Woman'] == 1].sum().reset_index()

,index,0
0,Woman,707
1,Parent,39
2,Pet Owner,341
3,Gamer,24
4,Student,684
5,Athletic,338
6,Foodie,40
7,Shopper,679


#### Parent

In [30]:
user_categories[user_categories['Parent'] == 1].sum().reset_index()

,index,0
0,Woman,39
1,Parent,46
2,Pet Owner,23
3,Gamer,3
4,Student,29
5,Athletic,19
6,Foodie,2
7,Shopper,35


#### Pet Owner

In [31]:
user_categories[user_categories['Pet Owner'] == 1].sum().reset_index()

,index,0
0,Woman,341
1,Parent,23
2,Pet Owner,360
3,Gamer,16
4,Student,332
5,Athletic,171
6,Foodie,20
7,Shopper,332


#### Gamer

In [32]:
user_categories[user_categories['Gamer'] == 1].sum().reset_index()

,index,0
0,Woman,24
1,Parent,3
2,Pet Owner,16
3,Gamer,27
4,Student,18
5,Athletic,17
6,Foodie,1
7,Shopper,21


#### Student

In [33]:
user_categories[user_categories['Student'] == 1].sum().reset_index()

,index,0
0,Woman,684
1,Parent,29
2,Pet Owner,332
3,Gamer,18
4,Student,693
5,Athletic,331
6,Foodie,40
7,Shopper,677


#### Athletic

In [34]:
athletic = user_categories[user_categories['Athletic'] == 1]
athletic.sum().reset_index()

,index,0
0,Woman,338
1,Parent,19
2,Pet Owner,171
3,Gamer,17
4,Student,331
5,Athletic,346
6,Foodie,12
7,Shopper,330


#### Foodie

In [35]:
foodie = user_categories[user_categories['Foodie'] == 1]
foodie.sum().reset_index()

,index,0
0,Woman,40
1,Parent,2
2,Pet Owner,20
3,Gamer,1
4,Student,40
5,Athletic,12
6,Foodie,41
7,Shopper,38


#### Shopper

In [36]:
shopper = user_categories[user_categories['Shopper'] == 1]
shopper.sum().reset_index()

,index,0
0,Woman,679
1,Parent,35
2,Pet Owner,332
3,Gamer,21
4,Student,677
5,Athletic,330
6,Foodie,38
7,Shopper,689


### Most common visits for each category

In [37]:
women_users=users["user_id"].loc[users["Woman"] == True ]
parent_users=users["user_id"].loc[users["Parent"] == True ]
petOwner_users=users["user_id"].loc[users["Pet Owner"] == True ]
gamer_users=users["user_id"].loc[users["Gamer"] == True ]
student_users=users["user_id"].loc[users["Student"] == True ]
athletic_users=users["user_id"].loc[users["Athletic"] == True ]
variety_users=users["user_id"].loc[users["variety"] == True ]
foodie_users=users["user_id"].loc[users["Foodie"] == True ]
shopper_users=users["user_id"].loc[users["Shopper"] == True ]

Women most common visits:

In [38]:
wm_ato = pd.Series(women_users)
wm_dt = wm_ato.to_frame()
wm = pd.merge(wm_dt,users)
wm_visits = wm[["category"]]
wm_visits["category"].value_counts()

Women's Store        709
University           685
Metro Station        682
Cafe                 682
Square               682
Supermarket          682
Clothing Store       681
Shopping Mall        679
Home Store           678
Park                 678
Cultural Center      677
Electronics Store    672
Gym                  671
Beach                669
Bakery               669
Shoe Store           668
Pet Store            643
Museum               625
Ice Cream Shop       571
Sandwich Place       539
Marina               524
Airport              443
Convenience Store    428
Bar                  428
Burgers              401
Kids Store           379
Restaurant           334
General              277
Playground           248
Cinema               202
Open Air Cinema      188
Gaming Cafe          161
Landmark             141
Waffle House         101
Taverna               69
Pastry Shop           51
Souvlaki              49
Game Store            45
Library               41
Pub/Beer              34


Parents most common visits:

In [39]:
prnt_ato = pd.Series(parent_users)
prnt_dt = prnt_ato.to_frame()
prnt = pd.merge(prnt_dt,users)
prnt_visits = prnt[["category"]]
prnt_visits["category"].value_counts()

Women's Store        41
Pet Store            40
Kids Store           40
Metro Station        36
Supermarket          36
Square               36
Home Store           36
Bakery               36
Cafe                 36
Clothing Store       36
Electronics Store    35
Cultural Center      35
Shopping Mall        35
Gym                  35
Museum               33
Shoe Store           32
Park                 32
Beach                31
University           30
Sandwich Place       26
Ice Cream Shop       26
Playground           25
Bar                  22
Marina               22
Burgers              20
Convenience Store    20
Restaurant           20
Airport              18
General              16
Cinema               13
Open Air Cinema      13
Gaming Cafe           8
Landmark              7
Waffle House          4
Pub/Beer              4
Library               3
Pastry Shop           3
Italian               2
Taverna               2
Game Store            2
Steakhouse            1
Bank            

Pet Owners most common visits:

In [40]:
pet_ato = pd.Series(petOwner_users)
pet_dt = pet_ato.to_frame()
pet_owners = pd.merge(pet_dt,users)
pet_owners = pet_owners[["category"]]
pet_owners["category"].value_counts()

Pet Store            360
Women's Store        346
Cafe                 333
Clothing Store       333
Square               333
University           333
Metro Station        333
Supermarket          333
Home Store           331
Shopping Mall        331
Gym                  331
Cultural Center      329
Park                 329
Electronics Store    328
Bakery               325
Shoe Store           325
Beach                322
Museum               307
Ice Cream Shop       269
Marina               267
Sandwich Place       262
Airport              220
Convenience Store    213
Burgers              206
Bar                  199
Kids Store           191
Restaurant           165
General              132
Playground           131
Open Air Cinema       97
Cinema                91
Gaming Cafe           72
Landmark              69
Waffle House          53
Taverna               34
Pastry Shop           25
Souvlaki              24
Library               22
Office                21
Game Store            20


Gamers most common visits:

In [41]:
gamer_ato = pd.Series(gamer_users)
gamer_dt = gamer_ato.to_frame()
gamers = pd.merge(gamer_dt,users)
gamers_visits = gamers[["category"]]
gamers_visits["category"].value_counts()

Gaming Cafe          27
Gym                  25
Women's Store        24
Pet Store            24
Metro Station        22
Supermarket          22
Square               22
Electronics Store    22
Home Store           22
Bakery               22
Cafe                 22
Cultural Center      21
Shoe Store           21
Clothing Store       21
Shopping Mall        20
Museum               19
Beach                19
Park                 19
University           18
Convenience Store    17
Sandwich Place       17
Ice Cream Shop       16
Marina               16
Burgers              12
Kids Store           11
Restaurant           11
Playground           10
Airport               9
General               7
Bar                   7
Open Air Cinema       7
Waffle House          6
Cinema                5
Pub/Beer              3
Landmark              3
Game Store            2
Pastry Shop           2
Bank                  1
Office                1
Library               1
Beauty Store          1
Steakhouse      

Students most common visits:

In [42]:
students_ato = pd.Series(student_users)
student_dt = students_ato.to_frame()
std = pd.merge(student_dt,users)
std_visits = std[["category"]]
std_visits["category"].value_counts()

University           693
Women's Store        691
Shopping Mall        679
Metro Station        679
Park                 679
Cafe                 679
Supermarket          679
Square               679
Clothing Store       678
Cultural Center      677
Home Store           676
Beach                673
Electronics Store    669
Gym                  666
Shoe Store           665
Bakery               665
Pet Store            635
Museum               626
Ice Cream Shop       575
Sandwich Place       534
Marina               530
Airport              443
Bar                  426
Convenience Store    422
Burgers              395
Kids Store           373
Restaurant           329
General              276
Playground           243
Cinema               198
Open Air Cinema      183
Gaming Cafe          156
Landmark             139
Waffle House         103
Taverna               69
Souvlaki              49
Pastry Shop           48
Game Store            43
Library               41
Pub/Beer              32


Athletics most common visits:

In [43]:
athl_ato = pd.Series(athletic_users)
athl_dt = athl_ato.to_frame()
athl = pd.merge(athl_dt,users)
athl_visits = athl[["category"]]
athl_visits["category"].value_counts()

Gym                  346
Women's Store        341
University           332
Clothing Store       331
Square               331
Cafe                 331
Supermarket          331
Metro Station        331
Shopping Mall        330
Park                 329
Cultural Center      329
Home Store           328
Bakery               327
Electronics Store    325
Shoe Store           325
Beach                324
Pet Store            315
Museum               302
Ice Cream Shop       277
Marina               256
Sandwich Place       250
Convenience Store    215
Airport              210
Bar                  196
Burgers              193
Kids Store           184
Restaurant           153
General              136
Playground           113
Cinema               104
Open Air Cinema       92
Gaming Cafe           79
Landmark              61
Waffle House          45
Taverna               30
Pastry Shop           23
Library               22
Souvlaki              22
Game Store            19
Pub/Beer              18


Shopper most common visits:

In [44]:
shop_ato = pd.Series(shopper_users)
shop_dt = shop_ato.to_frame()
shop = pd.merge(shop_dt,users)
shop_visits = shop[["category"]]
shop_visits["category"].value_counts()

Cafe                 18014
Supermarket          18014
Metro Station        17998
Square               17998
Clothing Store       17988
Women's Store        17976
Shopping Mall        17962
Park                 17929
Cultural Center      17912
Home Store           17897
Electronics Store    17780
University           17770
Beach                17718
Shoe Store           17672
Bakery               17666
Gym                  17451
Pet Store            16680
Museum               16553
Ice Cream Shop       15137
Sandwich Place       14280
Marina               14022
Airport              11846
Convenience Store    11495
Bar                  11450
Burgers              10678
Kids Store           10145
Restaurant            8890
General               7537
Playground            6640
Cinema                5453
Open Air Cinema       5046
Gaming Cafe           4249
Landmark              3880
Waffle House          2751
Taverna               1885
Pastry Shop           1366
Souvlaki              1353
G

Foodie most common visits:

In [57]:
fod_ato = pd.Series(foodie_users)
fod_dt = fod_ato.to_frame()
fod = pd.merge(fod_dt,users)
fod_visits = fod[["category"]]
fod_visits["category"].value_counts()

Sandwich Place       1160
Cafe                 1160
Supermarket          1160
Shopping Mall        1145
Clothing Store       1145
Square               1145
Beach                1145
Cultural Center      1145
Home Store           1145
University           1145
Park                 1145
Women's Store        1145
Metro Station        1145
Electronics Store    1135
Bakery               1130
Pet Store            1117
Shoe Store           1091
Restaurant           1067
Gym                  1062
Museum               1057
Burgers              1047
Marina                975
Ice Cream Shop        885
Bar                   851
Airport               840
Convenience Store     747
Kids Store            700
Taverna               562
General               529
Landmark              415
Souvlaki              398
Open Air Cinema       353
Playground            348
Cinema                323
Waffle House          231
Italian               208
Gaming Cafe           204
Steakhouse            192
Ethnic      

### Top 3 Days of each category

In [56]:
topDays=walkins["created"]
ato = pd.Series(topDays)
dt = ato.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dw_mapping = {
    0:"Monday",
    1:"Tuesday",
    2:"Wednesday",
    3:"Thursday" ,
    4:"Friday"  ,
    5:"Saturday",
    6:"Sunday"
}
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Friday       23024
Wednesday    22207
Thursday     22170
Saturday     21992
Tuesday      20083
Monday       16512
Sunday       15197
Name: Week_day, dtype: int64

Women

In [46]:
wm_merge= pd.merge(wm_ato,visits)
wm_days = wm_merge["created"].loc[(wm_merge["category"] == 'Women\'s Store') | (wm_merge["category"] =="Beauty Store")]
wm_days = pd.Series(wm_days)
dt= wm_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Saturday     1216
Monday       1099
Tuesday       984
Wednesday     886
Friday        819
Thursday      586
Sunday        327
Name: Week_day, dtype: int64

Parents

In [47]:
prnt_merge= pd.merge(prnt_ato,visits)
prnt_days = prnt_merge["created"].loc[(prnt_merge["category"] == 'Playground') | (prnt_merge["category"] =="Kids Store")]
wm_days = pd.Series(wm_days)
dt= prnt_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Friday       80
Tuesday      54
Thursday     43
Wednesday    38
Sunday       34
Saturday     25
Monday       25
Name: Week_day, dtype: int64

Pets Owners

In [50]:
pet_merge= pd.merge(pet_ato,visits)
pet_days = pet_merge["created"].loc[pet_merge["category"] == 'Pet Store' ]
pet_days = pd.Series(pet_days)
dt= pet_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Tuesday      461
Friday       452
Monday       303
Wednesday    289
Thursday     277
Saturday     255
Sunday       250
Name: Week_day, dtype: int64

Gamers

In [51]:
gamer_merge= pd.merge(gamer_ato,visits)
gamer_days = gamer_merge["created"].loc[gamer_merge["category"] == 'Gaming Cafe' ]
gamer_days = pd.Series(gamer_days)
dt= gamer_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Thursday     34
Saturday     31
Friday       31
Sunday       19
Tuesday      18
Wednesday     3
Monday        1
Name: Week_day, dtype: int64

Student

In [52]:
students_merge= pd.merge(students_ato,visits)
students_days = students_merge["created"].loc[students_merge["category"] == 'University' ]
students_days = pd.Series(students_days)
dt= students_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Thursday     3194
Tuesday      1511
Wednesday    1384
Saturday      437
Monday        359
Friday        303
Name: Week_day, dtype: int64

Athletic

In [53]:
athl_merge= pd.merge(athl_ato,visits)
athl_days = athl_merge["created"].loc[athl_merge["category"] == 'Gym' ]
athl_days = pd.Series(athl_days)
dt= athl_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Sunday       389
Tuesday      340
Thursday     328
Saturday     281
Friday       272
Wednesday    268
Monday       177
Name: Week_day, dtype: int64

Foodie

In [58]:
fod_merge= pd.merge(fod_ato,visits)
fod_days = fod_merge["created"].loc[(fod_merge["category"] == 'Souvlaki') | (fod_merge["category"] =="Taverna")
                                   | (fod_merge["category"] =="Steakhouse") | (fod_merge["category"] =="Italian")
                                   | (fod_merge["category"] =="Restaurant") | (fod_merge["category"] =="Ethnic")
                                   | (fod_merge["category"] =="Burgers")]
fod_days = pd.Series(fod_days)
dt= fod_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Saturday    1572
Sunday       997
Monday       633
Thursday     572
Friday       517
Tuesday      516
Name: Week_day, dtype: int64

Shopper

In [59]:
shop_merge= pd.merge(athl_ato,visits)
shop_days = shop_merge["created"].loc[(shop_merge["category"] == 'Women\'s Store') |
                                      (shop_merge["category"] == 'Mobile Phone Shop') |
                                      (shop_merge["category"] == 'Shoe Store') |
                                      (shop_merge["category"] == 'Clothing Store') |
                                      (shop_merge["category"] == 'Beauty Store') |
                                      (shop_merge["category"] == 'Game Store') |
                                      (shop_merge["category"] == 'Electronics Store') |
                                      (shop_merge["category"] == 'Kids Store') |
                                      (shop_merge["category"] == 'Shopping Mal') |
                                      (shop_merge["category"] == 'Home Store')]
shop_days = pd.Series(shop_days)
dt= shop_days.to_frame()
dt["created"] = pd.to_datetime(dt["created"])
dt["Week_day"] = dt["created"].dt.weekday.map(dw_mapping)
dt["Week_day"].value_counts()

Friday       1675
Tuesday      1640
Saturday     1627
Wednesday    1598
Thursday     1453
Sunday       1402
Monday       1214
Name: Week_day, dtype: int64